This notebook is used to process the flight schedule data.
Namely, we will simulate the nnumber of passenger on each flight. We will simply assume that it is normally distributed over a mean of 30 and a standard deviation of 50.

## Set-up

In [142]:
# import packages
import numpy as np 
import math
import pandas as pd 
import os 

In [143]:
# load the csv files
data_files = os.listdir("./data")

## Processing
We will add a column to each of the flight schedule we have and parse the date as appropriate.
Note that the current data file only includes the time but not the date. 

In [147]:
for file in data_files:
    df = pd.read_csv(f"./data/{file}")
    print(f"Processing file {file}")

    # add a column of integer passenger load per flight
    np.random.seed(2024)
    df['pass_load'] = np.random.normal(300,50,size =len(df))
    df['pass_load']  = df['pass_load'].apply(lambda x: math.floor(x))

    # parse the schedule time to add the date
    month = file[2:5]
    date = file[0:2]
    time_sch = pd.Series(f"2023 {month} {date} " + df['time_sch'])
    df['time_sch'] = pd.to_datetime(time_sch, format = "%Y %b %d %H:%M")

    # parsing the actual time
    # if file is on amsterdam
    if file[10:13] == "AMS":
        # the actual time either on time or delayed
        time_act = pd.Series(f"2023 {month} {date} " + df['time_act'])
        df['time_act'] = pd.to_datetime(time_act, format = "%Y %b %d %H:%M")
        df["time_diff"]  = df.apply(lambda x:(x['time_act'] - x['time_sch']).total_seconds(), axis = 1)
        # columns with negative time difference
        indices = df.query("time_diff < 0").index
        for ind in indices:
            df.loc[ind,"time_act"] = df.loc[ind,"time_act"] + pd.Timedelta(1, unit = "D")

    elif file[10:13] == "HKG":
        # parse the time
        time_act_t = df["time_act"].str.split(":")
        time_act_t = time_act_t.apply(lambda x: f"{x[0][-2:]}:{x[1][:2]}" if len(x) == 2 else "23:59")
        # parse the date
        time_act_d = df["time_act"].str.split("/")
        time_act_d = time_act_d.apply(lambda x: x[0][-2:] if len(x) > 2 else None)
        time_act_d = time_act_d.fillna(date)
        # concatenate the time and the date
        time_act = "2023 " + month + " " + time_act_d + " " + time_act_t
        df['time_act'] = pd.to_datetime(time_act, format = "%Y %b %d %H:%M")
        
    df = df.sort_values("time_sch").reset_index(drop = True)

    # save to new csv
    file_name = file[:-4]+"_processed.csv"
    print(file_name)
    df.to_csv(f"./data/{file_name}", index = False)


Processing file 20DEC2023_HKG.csv
20DEC2023_HKG_processed.csv
Processing file 21DEC2023_HKG.csv
21DEC2023_HKG_processed.csv
Processing file 19DEC2023_LHR.csv
19DEC2023_LHR_processed.csv
Processing file 19DEC2023_AMS.csv
19DEC2023_AMS_processed.csv
Processing file 22DEC2023_HKG.csv
22DEC2023_HKG_processed.csv
Processing file 19DEC2023_HKG.csv
19DEC2023_HKG_processed.csv
Processing file 22DEC2023_AMS.csv
22DEC2023_AMS_processed.csv
Processing file 21DEC2023_AMS.csv
21DEC2023_AMS_processed.csv
